In [ ]:
import os

import pandas as pd
import requests
from dotenv import load_dotenv

# This reads your .env file and puts variables into the environment
load_dotenv("../config.env")

# Get API key
API_KEY = os.getenv('MARKETSTACK_API_KEY')  
API_URL = 'https://api.marketstack.com/v2'

In [ ]:
TICKERS = ['AAPL', 'MSFT', 'AMZN', 'NVDA', 'GOOGL', 'META', 'TSLA']
LIMIT = 1000
OFFSET = 0
# START_DATE = '2020-01-01'
START_DATE = '2012-05-18'
END_DATE = '2025-11-04'
eod_api = f"{API_URL}/eod"

In [ ]:
# Define your parameters
params = {
    'access_key': API_KEY,
    'symbols': ','.join(TICKERS),  # Stock symbols
    'limit': LIMIT,                   # Number of results per page
    'offset': OFFSET,                    # Pagination offset (Number of results to skip)
    'sort': 'ASC',                 # Sort order
    'date_from': START_DATE,      # Start date
    'date_to': END_DATE         # End date
}

# Make the API call
response = requests.get(eod_api, params=params)

# Check if request was successful
if response.status_code == 200:
    data = response.json()
    print("API call successful!")
    print(f"Number of records: {len(data.get('data', []))}")
else:
    print(f"API call failed with status code: {response.status_code}")
    print(f"Error: {response.text}")

In [ ]:
data

In [ ]:
data['data']

In [ ]:
stock_data = data['data']

df = pd.DataFrame(stock_data)
df.head()

In [ ]:
final_df = df[['date', 'symbol', 'open', 'high', 'low', 'close', 'volume', 'adj_close']]
final_df.head()

In [ ]:
final_df.info()

In [ ]:
final_df.symbol.unique()

In [ ]:
final_df['date'] = pd.to_datetime(final_df['date']).dt.date
# convert to date objects
# final_df['date'] = final_df['date'].dt.date

final_df.head()

In [ ]:
final_df.to_parquet("../data/raw/stock_data.parquet", index=False)

In [ ]:
# import pyarrow as pa
# import pyarrow.parquet as pq

# # Create PyArrow table and save
# table = pa.Table.from_pandas(final_df)
# pq.write_table(table, "../data/raw/stock_data.parquet")

In [ ]:
for symbol in TICKERS:
    all_data = []
    offset = 0
    while True:
        resp = requests.get(
        eod_api,
        params={
           "access_key": API_KEY,
           "symbols": symbol,
           "date_from": START_DATE,
           "date_to": END_DATE,
           "limit": LIMIT,
           "offset": offset,
           "sort": "ASC"
        }
        ).json()
        all_data.extend(resp["data"])
        if len(all_data) >= resp["pagination"]["total"]:
            break
        offset += 1000
    df_symbol = pd.DataFrame(all_data)
    df_symbol["symbol"] = symbol
    df_symbol.to_parquet(f"../data/raw/{symbol}_eod.parquet")

In [ ]:
df = pd.read_parquet('../data/raw/GOOGL_eod.parquet')
df.shape

In [ ]:
df.date.min()